# Merging festival_wide with IMDB awards data

# First: Getting statistics on the festival_wide dataset:
- Documentation claims contains n=9348 unique films
- Just checking if that is the case

In [2]:
import pandas as pd

# load festival_data
df = pd.read_csv('data/1_film-dataset_festival-program_wide.csv', dtype={'imdb.id': str})

# check for Empty IDs
missing_ids = df['imdb.id'].isnull().sum()

# check for Duplicate IDs
# counts number of rows that share an ID with another row
duplicate_ids = df.duplicated(subset=['imdb.id'], keep=False).sum()

print(f"Total Rows: {len(df)}")
print(f"Rows with MISSING 'imdb.id': {missing_ids}")
print(f"Rows with DUPLICATE 'imdb.id': {duplicate_ids}")

Total Rows: 9348
Rows with MISSING 'imdb.id': 1497
Rows with DUPLICATE 'imdb.id': 1539


- We ignore rows with missing imdb.id, and will write our script to merge our data with the awards data to account for duplicate ids by checking if a given duplicate id row is actually the same film, or a unique film (with unique uid). If unique, we also treat as missing.

# Second: Performing the merge

In [4]:
import numpy as np

# set file names
primary_file = 'data/1_film-dataset_festival-program_wide.csv'
secondary_file = 'data/3_imdb-dataset_awards_long.csv'
output_file = 'data/merged_film_data.csv'
failed_ids_file = 'failed_ids.txt'

# read 'imdb.id' values as string to ensure matching works correctly
df_primary = pd.read_csv(primary_file, dtype={'imdb.id': str})
df_secondary = pd.read_csv(secondary_file, dtype={'imdb.id': str})

print(f"Primary rows (Films): {len(df_primary)}")
print(f"Secondary rows (Awards): {len(df_secondary)}")

# validate ids against unique.ids 
# if multiple rows share the same 'imdb.id' but have DIFFERENT 'unique.id's,
# it means different movies are incorrectly sharing an imdb.id
# thou shalt be treating these as null to prevent false matches.

# filter for rows that actually have an imdb.id
valid_ids_mask = df_primary['imdb.id'].notna()

# group by imdb.id and count how many unique unique.ids share that ID
collision_check = df_primary[valid_ids_mask].groupby('imdb.id')['unique.id'].nunique()

# identify imdb.ids that map to >1 unique unique.id
bad_ids = collision_check[collision_check > 1].index.tolist()

if bad_ids:
    # print for clariyt
    print(f"{len(bad_ids)} IMDb ids shared by different movies")
    
    # set imdb.id to None for these conflicting rows
    df_primary.loc[df_primary['imdb.id'].isin(bad_ids), 'imdb.id'] = np.nan
else:
    print("no id collisions found")

# merge datasets!
# using how='left' to treat the festival program primary
merged_df = pd.merge(
    df_primary, 
    df_secondary, 
    on='imdb.id', 
    how='left', 
    indicator=True 
)

# get stats for results
matches_df = merged_df[merged_df['_merge'] == 'both']
failures_df = merged_df[merged_df['_merge'] == 'left_only']

# count unique festival_films matched (gets same results as unique.id)
unique_films_matched = matches_df['imdb.id'].nunique()

# checking unique.ids to count unique films failed (doing unique,id on this bc changed imdb.id to null earlier if duplicate)
unique_films_failed = failures_df['unique.id'].nunique()

total_resulting_rows = len(merged_df)

print(f"Unique Films in Festival Data:   {df_primary['unique.id'].nunique()}")
print(f"Unique Films matched with Awards:  {unique_films_matched}")
print(f"Unique Films with no match:      {unique_films_failed}")
print(f"Total rows in merged df:        {total_resulting_rows}")

# keeping track of failures
if unique_films_failed > 0:
    # save the unique.id if the imdb.id is missing, otherwise the imdb.id
    with open(failed_ids_file, 'w') as f:
        f.write("unique.id,imdb.id\n") # Header
        
        # get unique pairs of unique.id/imdb.id from failures
        failed_pairs = failures_df[['unique.id', 'imdb.id']].drop_duplicates()
        
        for index, row in failed_pairs.iterrows():
            f.write(f"{row['unique.id']},{row['imdb.id']}\n")

# save everything
final_df = merged_df.drop(columns=['_merge'])
final_df.to_csv(output_file, index=False)

Primary rows (Films): 9348
Secondary rows (Awards): 106011
21 IMDb ids shared by different movies
Unique Films in Festival Data:   9348
Unique Films matched with Awards:  5770
Unique Films with no match:      3578
Total rows in merged df:        105266


In [8]:
final_df.head(20)

,unique.id,imdb.id,title.mixed,prod.year,length.min,length,prod.country.1.en,prod.country.2.en,prod.country.3.en,prod.country.4.en,...,sample.year.first,fest.show.delay.3year.and.more,award.institution,imdb.event.id,award.imdb.year,awards.nom.win,award,award.category,award.person,imdb.name.id
0,1,tt2917506,a story of children and film,2013.0,101.0,41 min. or longer,United Kingdom,NaN,NaN,NaN,...,2013,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,2012,0.0,"Asian Film Festival, Reggio Emilia",ev0003030,2014.0,Nominee,Best Film,NaN,Flora Lau,nm3408503
2,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,2012,0.0,Cannes Film Festival,ev0000147,2013.0,Nominee,Golden Camera,NaN,Flora Lau,nm3408503
3,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,2012,0.0,Cannes Film Festival,ev0000147,2013.0,Nominee,Un Certain Regard Award,NaN,Flora Lau,nm3408503
4,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,2012,0.0,Five Flavours Asian Film Festival,ev0003301,2013.0,Nominee,New Asian Cinema,Best Film,Flora Lau,nm3408503
5,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,2012,0.0,Golden Horse Film Festival,ev0000293,2013.0,Nominee,Golden Horse Award,Best Original Film Score,Patrick Jonsson,nm3486823
6,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,2012,0.0,Hong Kong Asian Film Festival,ev0002488,2013.0,Nominee,New Talent Award,NaN,Flora Lau,nm3408503
7,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,2012,0.0,"International Women Film Festival of Sale, Mor...",ev0002503,2013.0,Winner,Jury Prize,Best Actor,Kun Chen,nm1179512
8,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,2012,0.0,"International Women Film Festival of Sale, Mor...",ev0002503,2013.0,Nominee,Grand Prix,Best Film,Flora Lau,nm3408503
9,10,tt2852460,bends,2013.0,96.0,41 min. or longer,China,NaN,NaN,NaN,...,2012,0.0,Osaka Asian Film Festival,ev0002593,2014.0,Winner,Best Actress Award,NaN,Carina Lau,nm0490500
